In [10]:
import os

In [3]:
os.chdir('../')

In [11]:
%pwd

'd:\\Manav\\Projects\\Wine-Quality-Prediction'

In [12]:
from dataclasses import dataclass
from pathlib import Path
from urllib import request
from zipfile import ZipFile
from wineQuality.constants import *
from wineQuality.utils import read_yaml, create_directories, get_size
from wineQuality import logger

In [13]:

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path

In [14]:
class ConfigurationManager:
    def __init__(self,
                config_filepath= CONFIG_FILE_PATH,
                params_filepath= PARAMS_FILE_PATH,
                schemas_filepath= SCHEMAS_FILE_PATH):

                self.config = read_yaml(config_filepath)
                self.params = read_yaml(params_filepath)
                self.schemas = read_yaml(schemas_filepath)
                
                create_directories([self.config.artifacts_root])
                
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config

In [15]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file)
            logger.info(f'{filename} downloaded with following info: \n{headers}')
        else:
            logger.info(f'File already exists with size : {get_size(Path(self.config.local_data_file))}')
    
    def extract_zipfile(self):
        '''
        zip_file_path : str
        Extracts the zip file into data directory path
        Function returns None
        '''
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [16]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zipfile()
except Exception as e:
    raise e

[2024-02-29 20:21:24,254: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-29 20:21:24,256: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-29 20:21:24,259: INFO: common: yaml file: schemas.yaml loaded successfully]
[2024-02-29 20:21:24,260: INFO: common: created directory at: artifacts]
[2024-02-29 20:21:24,261: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-29 20:21:24,262: INFO: 1395391058: File already exists with size : ~ 21 KB]
